# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [24]:
a = data.groupby(["CustomerID","ProductName"]).sum()
display(a.head())

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [26]:
table = pd.pivot_table(data = a, values = "Quantity", index='ProductName',columns='CustomerID')
table = table.fillna(0)
table.head()


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [27]:
distances = pdist(table.T, 'euclidean')
distances_matrix = squareform(distances)
#display(distances_matrix)

distances = pd.DataFrame(1/(1 + distances_matrix), index=table.columns, columns=table.columns)
display(distances)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
464,0.082709,0.075255,0.080350,0.078187,0.078711,1.000000,0.074798,0.084626,0.080070,0.078187,...,0.004816,0.004687,0.004416,0.005038,0.004308,0.004533,0.004578,0.004363,0.004167,0.004347
477,0.074573,0.075956,0.076923,0.075025,0.075025,0.074798,1.000000,0.078977,0.075720,0.074573,...,0.004823,0.004688,0.004426,0.005037,0.004321,0.004534,0.004589,0.004366,0.004168,0.004338
639,0.083020,0.076435,0.080634,0.082403,0.082403,0.084626,0.078977,1.000000,0.085983,0.083020,...,0.004816,0.004682,0.004412,0.005035,0.004308,0.004526,0.004581,0.004363,0.004176,0.004335
649,0.081503,0.077674,0.082100,0.077171,0.078187,0.080070,0.075720,0.085983,1.000000,0.079792,...,0.004810,0.004676,0.004414,0.005035,0.004314,0.004528,0.004585,0.004369,0.004172,0.004329


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [28]:
similarities = distances.loc[33].sort_values(ascending=False)[1:].head(5)
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [29]:
d = a.loc[similarities.index]
d.head()

SalesID  ProductID  Quantity
CustomerID ProductName                                             
264        Apricots - Halves           3730301        278         1
           Apricots Fresh              5035449        158         1
           Bacardi Breezer - Tropical   756802         49         1
           Bagel - Plain               6728381        262         1
           Banana - Leaves             6298459        271         1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [30]:
e = d.groupby(["ProductName"]).agg({"Quantity":'sum'}).sort_values(by='Quantity', ascending=False)
e.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [34]:
f = e.merge(table, left_on='ProductName', right_on='ProductName')
f = f[["Quantity",33]]
f.head()

,Quantity,33
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Towels - Paper / Kraft,3,1.0
Soup - Campbells Bean Medley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
def recommendations(idx):
    similarities = distances.loc[idx].sort_values(ascending=False)[1:].head()
    d = a.loc[similarities.index]
    e = d.groupby(["ProductName"]).agg({"Quantity":'sum'}).sort_values(by='Quantity', ascending=False)
    f = df6.merge(piv, left_on='ProductName', right_on='ProductName')
    df7 = df7[["Quantity",idx]]
    df7 = df7.loc[df7[idx].isin([0])].head(5)
    return df7.index

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [37]:
dist = pdist(table.T, 'cosine')
dist_matrix10 = squareform(dist)
dist = pd.DataFrame(1/(1 + dist_matrix10), index=table.columns, columns=table.columns)
display(dist)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
464,0.541054,0.519193,0.525368,0.523986,0.534073,1.000000,0.528047,0.546574,0.528571,0.527240,...,0.537619,0.544446,0.534295,0.535278,0.526394,0.542873,0.520835,0.533856,0.528842,0.552221
477,0.522016,0.556945,0.538967,0.536792,0.542308,0.528047,1.000000,0.547544,0.534712,0.536302,...,0.543155,0.541855,0.545908,0.530540,0.544731,0.540381,0.534221,0.535601,0.527943,0.532238
639,0.530783,0.516729,0.514932,0.540890,0.547743,0.546574,0.547544,1.000000,0.555194,0.548386,...,0.539687,0.538500,0.528522,0.532803,0.527813,0.533190,0.525789,0.535704,0.548943,0.531706
649,0.533096,0.536350,0.537046,0.516901,0.530459,0.528571,0.534712,0.555194,1.000000,0.538236,...,0.529649,0.528778,0.530655,0.531114,0.537025,0.535224,0.531918,0.544926,0.539210,0.519657
